<a href="https://colab.research.google.com/github/surajjorwar/Book-Recommendation-System/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing fuzzywuzzy library
!pip install fuzzywuzzy

In [2]:
#importing library's
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz, process
from IPython.display import display, Image, HTML

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#loading csv files
book = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Books.csv')
rating = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Ratings.csv')
users = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Users.csv')

<ipython-input-3-b36acff5a32a>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Books.csv')
